<a href="https://colab.research.google.com/github/harshakoneru98/city_watch/blob/main/modelling/Crime_Rate_Forecasting_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import tqdm
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
ROOT_PATH = "/content/drive/MyDrive/DSCI 560/Datasets/Cleaned_data/"